### Ingest results.json file

In [0]:
# Create a text widget for data source input with an empty default value
dbutils.widgets.text("p_data_source", "")

# Retrieve the value of the data source widget
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
# Create a text widget for file date input with a default value of "2021-03-28"
dbutils.widgets.text("p_file_date", "2021-03-28")

# Retrieve the value of the file date widget
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
# Running the configuration notebook to set up environment variables and configurations
%run "../includes/configuration"

In [0]:
# Run the common functions notebook to make its functions available in the current notebook
%run "../includes/common_functions"

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
# Importing necessary classes from pyspark.sql.types to define schema for DataFrame
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [0]:
# Define the schema for the results DataFrame
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [0]:
# Read the JSON file into a DataFrame using the predefined schema
results_df = spark.read \
    .schema(results_schema) \
    .json(f"{raw_folder_path}/{v_file_date}/results.json")

##### Step 2 - Rename columns and add new columns

In [0]:
# Import the lit function from pyspark.sql.functions to add constant columns
from pyspark.sql.functions import lit

In [0]:
# Rename columns and add constant columns for data source and file date
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("data_source", lit(v_data_source)) \
                                    .withColumn("file_date", lit(v_file_date))

In [0]:
# Add ingestion date to the DataFrame
results_with_ingestion_date_df = add_ingestion_date(results_with_columns_df)

##### Step 3 - Drop the unwanted column

In [0]:
# Import the col function from pyspark.sql.functions to reference DataFrame columns
from pyspark.sql.functions import col

In [0]:
# Drop the 'statusId' column from the DataFrame
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))

De-dupe the dataframe

In [0]:
# Remove duplicate rows based on 'race_id' and 'driver_id' columns
results_deduped_df = results_final_df.dropDuplicates(['race_id', 'driver_id'])

##### Step 4 - Write to output to processed container in parquet format

In [0]:
# Define the merge condition for the Delta table
merge_condition = "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"

# Merge the deduplicated DataFrame into the Delta table
merge_delta_data(results_deduped_df, 'f1_processed', 'results', processed_folder_path, merge_condition, 'race_id')

In [0]:
# Exit the notebook with a success message
dbutils.notebook.exit("Success")

In [0]:
%sql
-- Count the total number of records in the results table
SELECT COUNT(1)
FROM f1_processed.results;

In [0]:
%sql
-- Select race_id, driver_id, and the count of records for each combination
SELECT race_id, driver_id, COUNT(1) 
FROM f1_processed.results
GROUP BY race_id, driver_id -- Group by race_id and driver_id to aggregate the counts
HAVING COUNT(1) > 1 -- Filter to include only combinations with more than one record
ORDER BY race_id, driver_id DESC; -- Order the results by race_id in ascending order and driver_id in descending order

In [0]:
%sql
-- Select all columns from the results table for a specific race and driver
SELECT * 
FROM f1_processed.results 
WHERE race_id = 540 
  AND driver_id = 229;